<a href="https://colab.research.google.com/github/benshindel/journal-ranking/blob/main/journalrank3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import time
from tqdm.notebook import tqdm
import datetime
from google.colab import files # Import files for direct download

# --- Include all your original functions here (find_source_id, extract_paper_info, save_to_excel) ---
# ---  `find_source_id`, `extract_paper_info`, and `save_to_excel` remain UNCHANGED  ---
# --- `fetch_papers` and `fetch_journal_papers_spreadsheet` are MODIFIED to accept year(s) as input ---


# Function to find the source ID for a journal (NO CHANGE)
def find_source_id(journal_name, email):
    headers = {
        "User-Agent": f"Python Script ({email})"
    }

    # Search for the journal by name
    search_url = "https://api.openalex.org/sources"
    params = {
        "search": journal_name,
        "per_page": 5
    }

    print(f"Searching for journal: {journal_name}")
    response = requests.get(search_url, params=params, headers=headers)

    if response.status_code != 200:
        print(f"Error searching for journal: {response.status_code}")
        print(response.text)
        return None

    data = response.json()
    results = data.get("results", [])

    if not results:
        print(f"No journals found matching '{journal_name}'")
        return None

    # Display the top results
    print(f"Found {len(results)} potential matches:")
    for i, source in enumerate(results):
        print(f"{i+1}. {source.get('display_name')} - ID: {source.get('id')} - Publisher: {source.get('host_organization')}")

    # For automation, we'll take the first result if it's an exact match
    for source in results:
        if source.get("display_name").lower() == journal_name.lower():
            source_id = source.get("id")
            print(f"Selected exact match: {source.get('display_name')} (ID: {source_id})")
            return source_id

    # If no exact match, use the first result
    source_id = results[0].get("id")
    print(f"Selected best match: {results[0].get('display_name')} (ID: {source_id})")
    return source_id

# Function to extract relevant fields from paper metadata (NO CHANGE)
def extract_paper_info(papers):
    extracted_data = []

    for paper in tqdm(papers, desc="Processing papers"):
        # Extract basic information
        paper_info = {
            "title": paper.get("title", ""),
            "doi": paper.get("doi", ""),
            "publication_date": paper.get("publication_date", ""),
            "publication_year": paper.get("publication_year", ""),
            "open_access": paper.get("open_access", {}).get("is_oa", False),
            "cited_by_count": paper.get("cited_by_count", 0),
            "url": paper.get("primary_location", {}).get("landing_page_url", ""),
            "volume": paper.get("biblio", {}).get("volume", ""),
            "issue": paper.get("biblio", {}).get("issue", ""),

            # Added new metadata fields
            "type": paper.get("type", ""),  # Article type (e.g., article, review, etc.)
            "is_retracted": paper.get("is_retracted", False),
            "is_paratext": paper.get("is_paratext", False),
            "journal": paper.get("primary_location", {}).get("source", {}).get("display_name", ""),
            "journal_issn": paper.get("primary_location", {}).get("source", {}).get("issn_l", ""),
            "publisher": paper.get("primary_location", {}).get("host_organization_name", ""),
            "language": paper.get("language", ""),
            "countries": ", ".join([country.get("display_name", "") for country in paper.get("countries_distinct_names", [])]),
            "alternate_urls": ", ".join([loc.get("landing_page_url", "") for loc in paper.get("locations", [])[:3] if loc.get("landing_page_url")]),
            "topics": ", ".join([topic.get("display_name", "") for topic in paper.get("topics", [])[:5]]),
            "open_access_status": paper.get("open_access", {}).get("oa_status", ""),
            "open_access_url": paper.get("open_access", {}).get("oa_url", ""),
            "corresponding_author": "",
            "corresponding_institution": "",
            "citations_per_year": ", ".join([f"{year}: {count}" for year, count in paper.get("counts_by_year", [{}])[:5]]),
            "referenced_works_count": paper.get("referenced_works_count", 0),
            "grants": ", ".join([grant.get("funder_display_name", "") for grant in paper.get("grants", [])[:3]]),
        }

        # Extract authors (up to 5)
        authors = paper.get("authorships", [])
        author_names = []
        author_institutions = []

        # Look for corresponding author info
        for i, author in enumerate(authors):
            name = author.get("author", {}).get("display_name", "")
            if name:
                author_names.append(name)

            institutions = []
            for inst in author.get("institutions", []):
                inst_name = inst.get("display_name", "")
                if inst_name:
                    institutions.append(inst_name)

            if institutions:
                author_institutions.append("; ".join(institutions))

            # Check if this is a corresponding author
            if author.get("is_corresponding", False):
                paper_info["corresponding_author"] = name
                paper_info["corresponding_institution"] = "; ".join(institutions) if institutions else ""

        paper_info["authors"] = ", ".join(author_names[:5])
        paper_info["institutions"] = " | ".join(author_institutions[:5])
        paper_info["author_count"] = len(authors)

        # Extract concepts/keywords (up to 5 main ones)
        concepts = paper.get("concepts", [])
        concept_names = []

        for concept in sorted(concepts, key=lambda x: x.get("score", 0), reverse=True)[:5]:
            score = concept.get("score", 0)
            name = concept.get("display_name", "")
            if name:
                concept_names.append(f"{name} ({score:.2f})")

        paper_info["concepts"] = ", ".join(concept_names)

        # Extract abstract
        paper_info["abstract"] = paper.get("abstract_inverted_index", {})
        if paper_info["abstract"]:
            try:
                # Convert inverted index to text (OpenAlex specific format)
                words = list(paper_info["abstract"].keys())
                positions = {}

                for word, pos_list in paper_info["abstract"].items():
                    for pos in pos_list:
                        positions[pos] = word

                abstract_text = " ".join([positions.get(i, "") for i in range(max(positions.keys()) + 1)])
                paper_info["abstract"] = abstract_text
            except:
                paper_info["abstract"] = "Error processing abstract"
        else:
            paper_info["abstract"] = ""

        # Get mesh terms if available (medical papers)
        mesh_terms = []
        for mesh in paper.get("mesh", [])[:5]:
            qualifier = mesh.get("qualifier_name", "")
            descriptor = mesh.get("descriptor_name", "")
            if descriptor:
                term = descriptor
                if qualifier:
                    term += f" ({qualifier})"
                mesh_terms.append(term)

        paper_info["mesh_terms"] = ", ".join(mesh_terms)

        extracted_data.append(paper_info)

    return extracted_data

# Function to save data to Excel in Colab's local filesystem AND provide download link (NO CHANGE)
def save_to_excel(data, filepath, journal="ACS_Journals_YEAR_PAPERS", year="YEAR"): # Generic journal name, updated default filename prefix - year can now be string
    """
    Save paper data to Excel in Colab's local filesystem AND provide download link.

    Parameters:
    -----------
    data : list
        List of paper information dictionaries
    filepath : str
        Filepath to save the Excel file to within Colab.
    journal : str
        Journal name prefix for filename (now generic "ACS_Journals_YEAR_PAPERS")
    year : str or int
        Publication year or year range for filename (can be string now)

    Returns:
    --------
    str: The filepath where the Excel file was saved.
    """
    # Convert to DataFrame
    df = pd.DataFrame(data)

    # Save to Excel in Colab filesystem
    df.to_excel(filepath, index=False, engine='openpyxl')
    print(f"File saved to Colab filesystem: {filepath}")

    # --- ADDED: Download the file immediately ---
    try:
        files.download(filepath) # Trigger download
        print(f"Downloaded file: {filepath}")
    except ImportError:
        print("`files.download` not available (not in Colab). File is saved in Colab filesystem but not automatically downloaded.")

    return filepath


# --- MODIFIED fetch_papers to fetch from a LIST of journals for SPECIFIED year(s) ---
def fetch_papers(journal_names, year, email="benshindel@gmail.com"): # year can now be single year or a LIST of years
    headers = {
        "User-Agent": f"Python Script ({email})"
    }

    all_journal_papers = [] # List to hold papers from ALL journals

    # Check if year is a list or a single year
    if isinstance(year, list):
        years_to_fetch = year
    else:
        years_to_fetch = [year] # If single year, make it a list for iteration

    for journal_name in journal_names: # Iterate through each journal name
        source_id = find_source_id(journal_name, email) # Find source ID for EACH journal

        if not source_id:
            print(f"Could not find source ID for journal '{journal_name}'. Skipping.")
            continue # Skip to the next journal if source ID not found

        for current_year in years_to_fetch: # Iterate through each year to fetch
            # Base URL for OpenAlex API
            base_url = "https://api.openalex.org/works"

            # Filter for publication year AND source ID
            filters = [
                f"publication_year:{current_year}",
                f"primary_location.source.id:{source_id}"
            ]
            filter_param = ",".join(filters)

            # Parameters for API request
            params = {
                "filter": filter_param,
                "per_page": 200 # Max per page
            }

            journal_results = [] # List to store results for CURRENT journal AND year
            page = 1
            total_pages = None

            print(f"Fetching papers from '{journal_name}' (ID: {source_id}) published in {current_year}...") # Updated print for year
            print(f"Using filter: {filter_param}")

            while total_pages is None or page <= total_pages:
                params["page"] = page
                response = requests.get(base_url, params=params, headers=headers)

                if response.status_code != 200:
                    print(f"Error: API returned status code {response.status_code}")
                    print(response.text)
                    break

                data = response.json()

                if total_pages is None:
                    total_count = data.get("meta", {}).get("count", 0)
                    total_pages = (total_count + params["per_page"] - 1) // params["per_page"]
                    print(f"Found {total_count} papers for '{journal_name}' in {current_year}. Fetching {total_pages} pages...") # Updated print for year

                    if total_count == 0:
                        print(f"No papers found for '{journal_name}' in {current_year} matching criteria.") # Updated print for year
                        break # No papers for this journal in this year, move to the next year

                journal_results.extend(data.get("results", []))
                print(f"Fetched page {page}/{total_pages} from '{journal_name}' in {current_year} ({len(journal_results)} papers so far)") # Updated print for year
                page += 1
                time.sleep(0.2) # Rate limiting

            print(f"Successfully fetched {len(journal_results)} papers from '{journal_name}' in {current_year}.") # Updated print for year
            all_journal_papers.extend(journal_results) # Add results for this journal and year to the overall list

    print(f"Total papers fetched from all journals and years: {len(all_journal_papers)}") # Updated total count message
    return all_journal_papers


# --- MODIFIED Main execution function for STEP 1: Generate and Save Spreadsheet for MULTIPLE journals and SPECIFIED year(s) ---
def fetch_journal_papers_spreadsheet(journal_names, year, email="benshindel@gmail.com", output_filepath="ACS_journals_YEAR_papers.xlsx"): # year can now be single year or list, generic filename with YEAR placeholder
    """
    Main function to fetch and process papers from a list of journals for given year(s), and save to Excel.

    Parameters:
    -----------
    journal_names : list
        List of journal names to fetch papers from.
    year : int or list
        Publication year or list of years to fetch papers from.
    email : str
        Email for user agent (default: "benshindel@gmail.com")
    output_filepath : str
        Filepath to save the initial spreadsheet (default: "ACS_journals_YEAR_papers.xlsx") - YEAR will be replaced

    Returns:
    --------
    str: Filepath of the saved spreadsheet.
    """
    # Step 1: Fetch papers from ALL journals in the list for specified year(s)
    all_papers = fetch_papers(journal_names, year, email) # Fetch for list of journals and year(s)

    if not all_papers:
        print("No data to process.")
        return None

    # Step 2: Extract relevant information
    paper_data = extract_paper_info(all_papers)

    # Step 3: Save to Excel in Colab filesystem AND download

    # Determine year string for filename - handle single year or year range in filename
    if isinstance(year, list):
        year_str = f"{min(year)}-{max(year)}" # e.g., 2019-2022 if years = [2019, 2020, 2021, 2022]
    else:
        year_str = str(year) # e.g., 2024 if year = 2024

    output_filepath_final = output_filepath.replace("YEAR", year_str) # Replace "YEAR" placeholder in filename
    excel_filepath = save_to_excel(paper_data, output_filepath_final, journal=f"ACS_Journals_{year_str}_Papers", year=year_str) # Generic journal name for filename, use year_str


    # Create DataFrame (optional, for summary)
    df = pd.DataFrame(paper_data)

    # Display summary
    print(f"\nSummary (Initial Spreadsheet - Papers from Multiple ACS Journals for Year(s) {year_str}):") # Updated summary message for year(s)
    print(f"- Journals: {', '.join(journal_names)}") # List journal names in summary
    print(f"- Year(s): {year_str}") # Show year(s) in summary
    print(f"- Papers retrieved: {len(all_papers)}")
    print(f"- Initial Spreadsheet saved to: {excel_filepath} AND Downloaded")
    print(f"- Data columns: {', '.join(df.columns)}")

     # Add type distribution summary
    if 'type' in df.columns:
        print("\nArticle Type Distribution (Initial Data):")
        type_counts = df['type'].value_counts()
        for type_name, count in type_counts.items():
            print(f"- {type_name}: {count} ({count/len(df)*100:.1f}%)")


    return excel_filepath # Return filepath


# --- EXECUTION CELL for STEP 1: Generate, Save, and Download Spreadsheet for MULTIPLE journals and SPECIFIED year(s) ---
journal_names_to_fetch = [
    "ACS applied materials & interfaces",
    "Nanotechnology",
    "Beilstein Journal of Nanotechnology"
] # List of journal names

years_to_fetch = [2023] # Specify year(s) as a list OR a single year like: years_to_fetch = 2024
# years_to_fetch = 2024 # Example for single year

initial_spreadsheet_filepath = fetch_journal_papers_spreadsheet(journal_names=journal_names_to_fetch, year=years_to_fetch, email="benshindel@gmail.com", output_filepath="ACS_journals_YEAR_papers.xlsx") # Pass year(s)

print(f"\nInitial spreadsheet generation and download completed. File saved at: {initial_spreadsheet_filepath}")

Searching for journal: ACS applied materials & interfaces
Found 1 potential matches:
1. ACS Applied Materials & Interfaces - ID: https://openalex.org/S164001016 - Publisher: https://openalex.org/P4310320006
Selected exact match: ACS Applied Materials & Interfaces (ID: https://openalex.org/S164001016)
Fetching papers from 'ACS applied materials & interfaces' (ID: https://openalex.org/S164001016) published in 2023...
Using filter: publication_year:2023,primary_location.source.id:https://openalex.org/S164001016
Found 5585 papers for 'ACS applied materials & interfaces' in 2023. Fetching 28 pages...
Fetched page 1/28 from 'ACS applied materials & interfaces' in 2023 (200 papers so far)
Fetched page 2/28 from 'ACS applied materials & interfaces' in 2023 (400 papers so far)
Fetched page 3/28 from 'ACS applied materials & interfaces' in 2023 (600 papers so far)
Fetched page 4/28 from 'ACS applied materials & interfaces' in 2023 (800 papers so far)
Fetched page 5/28 from 'ACS applied materials

Processing papers:   0%|          | 0/6516 [00:00<?, ?it/s]

File saved to Colab filesystem: ACS_journals_2023-2023_papers.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloaded file: ACS_journals_2023-2023_papers.xlsx

Summary (Initial Spreadsheet - Papers from Multiple ACS Journals for Year(s) 2023-2023):
- Journals: ACS applied materials & interfaces, Nanotechnology, Beilstein Journal of Nanotechnology
- Year(s): 2023-2023
- Papers retrieved: 6516
- Initial Spreadsheet saved to: ACS_journals_2023-2023_papers.xlsx AND Downloaded
- Data columns: title, doi, publication_date, publication_year, open_access, cited_by_count, url, volume, issue, type, is_retracted, is_paratext, journal, journal_issn, publisher, language, countries, alternate_urls, topics, open_access_status, open_access_url, corresponding_author, corresponding_institution, citations_per_year, referenced_works_count, grants, authors, institutions, author_count, concepts, abstract, mesh_terms

Article Type Distribution (Initial Data):
- article: 6269 (96.2%)
- review: 139 (2.1%)
- erratum: 58 (0.9%)
- paratext: 26 (0.4%)
- editorial: 19 (0.3%)
- retraction: 4 (0.1%)
- letter: 1 (0.0%)

In

In [ ]:
import pandas as pd
from google.colab import files # Import files for download in the cleaning step
import numpy as np # Import numpy for NaN check (though pd.isna works without explicit import now in recent pandas versions, it's good practice to import numpy if dealing with NaNs)


# --- SIMPLIFIED CLEAN DATAFRAME FUNCTION (Direct NaN Check - NO CHANGES) ---
def clean_dataframe(df, volume_column_name="volume", institution_column_name="institutions"):
    """
    Cleans a Pandas DataFrame by removing rows that have NaN in volume or institution columns.

    Args:
        df (pd.DataFrame): The Pandas DataFrame to clean.
        volume_column_name (str): The name of the volume column.
        institution_column_name (str): The name of the institution column.

    Returns:
        pd.DataFrame: The cleaned Pandas DataFrame.
    """

    print("Cleaning DataFrame...")

    if df.empty:
        print("DataFrame is empty, nothing to clean.")
        return df

    rows_to_drop_indices = []

    for index, row in df.iterrows():
        is_volume_blank = False
        is_institution_blank = False

        # Check for NaN in volume column
        if volume_column_name in df.columns:
            if pd.isna(row[volume_column_name]): # Direct NaN check using pd.isna()
                is_volume_blank = True

        # Check for NaN in institution column
        if institution_column_name in df.columns:
            if pd.isna(row[institution_column_name]): # Direct NaN check using pd.isna()
                is_institution_blank = True

        # Delete row if volume OR institution is NaN
        if is_volume_blank or is_institution_blank:
            rows_to_drop_indices.append(index)
            # print(f"Row {index} marked for deletion (NaN in volume or institution).") # Optional: Minimal deletion message

    if rows_to_drop_indices:
        print(f"Identified {len(rows_to_drop_indices)} rows to delete from DataFrame (NaN values).")
        cleaned_df = df.drop(index=rows_to_drop_indices)
        print(f"DataFrame cleaned. {len(rows_to_drop_indices)} rows removed.")
        return cleaned_df
    else:
        print("No rows found to delete based on criteria (NaN values).")
        return df


# Main function for cleaning the GENERATED spreadsheet file (NO CHANGES)
def clean_generated_spreadsheet(input_filepath="ACS_journals_2023-2023_papers.xlsx", output_filepath="cleaned_papers.xlsx"): # Updated default input filename
    """
    Cleans the Excel spreadsheet generated by the first step.

    Args:
        input_filepath (str): Filepath of the initial spreadsheet.
        output_filepath (str): Filepath to save the cleaned spreadsheet.

    Returns:
        str: Filepath to the cleaned spreadsheet.
    """
    try:
        print(f"Loading spreadsheet for cleaning from: {input_filepath}")
        initial_df = pd.read_excel(input_filepath)

        # Clean the DataFrame
        cleaned_df = clean_dataframe(initial_df)

        # Save the cleaned DataFrame to a NEW Excel file
        cleaned_filepath = cleaned_df.to_excel(output_filepath, index=False, engine='openpyxl') # Save and get None return

        cleaned_filepath = output_filepath # Correctly set cleaned_filepath to the output path

        print(f"Cleaned spreadsheet saved to: {cleaned_filepath}")

        # Optional: Download the cleaned spreadsheet
        try:
            files.download(output_filepath)
            print(f"Downloaded cleaned spreadsheet: {output_filepath}")
        except ImportError:
            print("`files.download` not available (not in Colab). File is saved in Colab filesystem.")

        return cleaned_filepath

    except FileNotFoundError:
        print(f"Error: Input file not found at: {input_filepath}. Make sure you ran the first code block and the file was saved.")
        return None
    except Exception as e:
        print(f"An error occurred during spreadsheet cleaning: {e}")
        return None


# --- EXECUTION CELL for STEP 2: Clean the Spreadsheet generated in Step 1 (NO CHANGES) ---
cleaned_spreadsheet_filepath = clean_generated_spreadsheet(input_filepath="ACS_journals_2023-2023_papers.xlsx", output_filepath="cleaned_papers.xlsx") # Updated default input filename

if cleaned_spreadsheet_filepath:
    print(f"\nSpreadsheet cleaning completed. Cleaned file saved at: {cleaned_spreadsheet_filepath}")
else:
    print("\nSpreadsheet cleaning process failed. See error messages above.")

Loading spreadsheet for cleaning from: ACS_journals_2023-2023_papers.xlsx
Cleaning DataFrame...
Identified 534 rows to delete from DataFrame (NaN values).
DataFrame cleaned. 534 rows removed.
Cleaned spreadsheet saved to: cleaned_papers.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloaded cleaned spreadsheet: cleaned_papers.xlsx

Spreadsheet cleaning completed. Cleaned file saved at: cleaned_papers.xlsx


In [ ]:
import pandas as pd
from google.colab import files # For downloading the labeled spreadsheet

# --- CODE BLOCK 3: Labeling Spreadsheet with "Top Institution" Column ---
def label_top_institution(input_filepath="cleaned_papers.xlsx", output_filepath="labeled_papers.xlsx", top_institution_strings=["Harvard University", "Stanford University", "Massachusetts Institute of Technology", "	Tsinghua University", "University of Oxford", "Princeton University", "University of Cambridge", "Johns Hopkins University", "National University of Singapore"]):
    """
    Adds a "Top_Institution" column to the cleaned spreadsheet, labeling "YES" if
    any of the specified top institution strings are found in the 'institutions' column,
    and "NO" otherwise.

    Args:
        input_filepath (str): Filepath to the cleaned spreadsheet (default: "cleaned_papers.xlsx").
        output_filepath (str): Filepath to save the labeled spreadsheet (default: "labeled_papers.xlsx").
        top_institution_strings (list): List of strings representing top institutions to search for.

    Returns:
        str: Filepath to the labeled spreadsheet.
    """
    try:
        print(f"Loading cleaned spreadsheet for labeling from: {input_filepath}")
        cleaned_df = pd.read_excel(input_filepath)

        if cleaned_df.empty:
            print("Warning: Cleaned DataFrame is empty. No labeling to perform.")
            return None  # Or return input_filepath if you still want to save it as labeled

        # Initialize the new column with "NO" as default
        cleaned_df['Top_Institution'] = "NO"

        # Convert 'institutions' column to string type to handle potential mixed types and avoid errors during string operations
        cleaned_df['institutions'] = cleaned_df['institutions'].astype(str)

        # Iterate through each row and check for top institution strings
        for index, row in cleaned_df.iterrows():
            institutions_text = row['institutions'].lower() # Convert to lowercase for case-insensitive matching
            for top_institution_string in top_institution_strings:
                if top_institution_string.lower() in institutions_text: # Case-insensitive check
                    cleaned_df.at[index, 'Top_Institution'] = "YES" # Use .at for efficient value setting
                    break # No need to check other strings once a match is found

        # Save the labeled DataFrame to a new Excel file
        labeled_filepath = cleaned_df.to_excel(output_filepath, index=False, engine='openpyxl') # Save and get None return
        labeled_filepath = output_filepath # Correctly set labeled_filepath

        print(f"Labeled spreadsheet saved to: {labeled_filepath}")

        # Optional: Download the labeled spreadsheet
        try:
            files.download(output_filepath)
            print(f"Downloaded labeled spreadsheet: {output_filepath}")
        except ImportError:
            print("`files.download` not available (not in Colab). File is saved in Colab filesystem.")

        return labeled_filepath

    except FileNotFoundError:
        print(f"Error: Input file not found at: {input_filepath}. Make sure you ran Code Block 2 and the file was saved.")
        return None
    except Exception as e:
        print(f"An error occurred during spreadsheet labeling: {e}")
        return None


# --- EXECUTION CELL for STEP 3: Label Spreadsheet and Download ---
labeled_spreadsheet_filepath = label_top_institution(input_filepath="cleaned_papers.xlsx", output_filepath="labeled_papers.xlsx")

if labeled_spreadsheet_filepath:
    print(f"\nSpreadsheet labeling completed. Labeled file saved at: {labeled_spreadsheet_filepath}")
else:
    print("\nSpreadsheet labeling process failed. See error messages above.")

Loading cleaned spreadsheet for labeling from: cleaned_papers.xlsx
Labeled spreadsheet saved to: labeled_papers.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloaded labeled spreadsheet: labeled_papers.xlsx

Spreadsheet labeling completed. Labeled file saved at: labeled_papers.xlsx


In [ ]:
import pandas as pd
from google.colab import files # For downloading the journal ratios spreadsheet

# --- CODE BLOCK 6: Calculate and Output Journal YEARLY Top Institution Ratios WITH COUNTS ---
def calculate_journal_yearly_top_institution_ratios(input_filepath="labeled_papers.xlsx", output_filepath="journal_yearly_ratios_with_counts.xlsx"):
    """
    Calculates the ratio of papers with "Top_Institution" = "YES" for each journal
    AND each year present in the labeled spreadsheet. Outputs results to a new spreadsheet
    with years as columns, including total paper counts and YES counts.
    Also adds total columns at the end for each journal across all years.

    Args:
        input_filepath (str): Filepath to the labeled spreadsheet (default: "labeled_papers.xlsx").
        output_filepath (str): Filepath to save the journal yearly ratios spreadsheet (default: "journal_yearly_ratios_with_counts.xlsx").

    Returns:
        str: Filepath to the journal yearly ratios spreadsheet.
    """
    try:
        print(f"Loading labeled spreadsheet for yearly ratio and count calculation from: {input_filepath}")
        labeled_df = pd.read_excel(input_filepath)

        if labeled_df.empty:
            print("Warning: Labeled DataFrame is empty. Cannot calculate ratios or counts.")
            return None

        journal_names = labeled_df['journal'].unique() # Get unique journal names
        years = sorted(labeled_df['publication_year'].unique()) # Get unique years and sort them

        yearly_ratios_data = [] # List to store data for each journal (will be rows in output)

        for journal_name in journal_names:
            journal_ratios = {'Journal': journal_name} # Dictionary to store ratios and counts for each year for this journal
            total_top_institution_papers_all_years = 0 # Initialize total YES counts for all years
            total_journal_papers_all_years = 0 # Initialize total paper counts for all years

            for year in years:
                journal_year_df = labeled_df[(labeled_df['journal'] == journal_name) & (labeled_df['publication_year'] == year)]
                total_journal_year_papers = len(journal_year_df)
                top_institution_papers = len(journal_year_df[journal_year_df['Top_Institution'] == "YES"])

                ratio = 0 # Default ratio
                if total_journal_year_papers > 0:
                    ratio = top_institution_papers / total_journal_year_papers

                journal_ratios[f'{year}_Ratio'] = ratio # Store ratio, use year_Ratio as column name
                journal_ratios[f'{year}_TotalPapers'] = total_journal_year_papers # Store total papers count
                journal_ratios[f'{year}_YesCount'] = top_institution_papers # Store YES count

                total_top_institution_papers_all_years += top_institution_papers # Accumulate YES counts
                total_journal_papers_all_years += total_journal_year_papers # Accumulate total paper counts

            # Calculate total ratio across all years for this journal
            total_ratio_all_years = 0
            if total_journal_papers_all_years > 0:
                total_ratio_all_years = total_top_institution_papers_all_years / total_journal_papers_all_years

            # Add total columns to the journal_ratios dictionary
            journal_ratios['Total_Ratio'] = total_ratio_all_years
            journal_ratios['Total_TotalPapers'] = total_journal_papers_all_years
            journal_ratios['Total_YesCount'] = total_top_institution_papers_all_years

            yearly_ratios_data.append(journal_ratios) # Add journal's yearly ratios and counts to the list

        yearly_ratios_df = pd.DataFrame(yearly_ratios_data) # Create DataFrame from yearly ratios data
        yearly_ratios_df = yearly_ratios_df.set_index('Journal') # Set 'Journal' column as index for better readability

        # Save the journal yearly ratios DataFrame to a new Excel file
        ratios_filepath = yearly_ratios_df.to_excel(output_filepath, engine='openpyxl') # Index is now Journal, so index=True is default for index save
        ratios_filepath = output_filepath # Correctly set ratios_filepath

        print(f"Journal yearly and total top institution ratios and counts saved to: {ratios_filepath}")

        # Optional: Download the journal yearly ratios spreadsheet
        try:
            files.download(output_filepath)
            print(f"Downloaded journal yearly ratios spreadsheet: {output_filepath}")
        except ImportError:
            print("`files.download` not available (not in Colab). File is saved in Colab filesystem.")

        # Print ratios and counts to console in a table-like format
        print("\nJournal Yearly and Total Top Institution Ratios and Counts:")
        print(yearly_ratios_df.to_string(float_format="%.3f")) # Use to_string for DataFrame print, format floats to 3 decimals

        return ratios_filepath

    except FileNotFoundError:
        print(f"Error: Input file not found at: {input_filepath}. Make sure you ran Code Block 3 and the file was saved.")
        return None
    except Exception as e:
        print(f"An error occurred during yearly ratio and count calculation: {e}")
        return None


# --- EXECUTION CELL for STEP 6: Calculate and Output Journal YEARLY Ratios WITH COUNTS ---
journal_yearly_ratios_filepath = calculate_journal_yearly_top_institution_ratios(input_filepath="labeled_papers.xlsx", output_filepath="journal_yearly_ratios_with_counts.xlsx")

if journal_yearly_ratios_filepath:
    print(f"\nJournal yearly and total ratio and count calculation completed. Yearly and total ratios and counts file saved at: {journal_yearly_ratios_filepath}")
else:
    print("\nJournal yearly and total ratio and count calculation process failed. See error messages above.")

Loading labeled spreadsheet for yearly ratio and count calculation from: labeled_papers.xlsx
Journal yearly and total top institution ratios and counts saved to: journal_yearly_ratios_with_counts.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloaded journal yearly ratios spreadsheet: journal_yearly_ratios_with_counts.xlsx

Journal Yearly and Total Top Institution Ratios and Counts:
                                     2023_Ratio  2023_TotalPapers  2023_YesCount  Total_Ratio  Total_TotalPapers  Total_YesCount
Journal                                                                                                                         
ACS Applied Materials & Interfaces        0.015              5071             76        0.015               5071              76
Nanotechnology                            0.010               811              8        0.010                811               8
Beilstein Journal of Nanotechnology       0.010               100              1        0.010                100               1

Journal yearly and total ratio and count calculation completed. Yearly and total ratios and counts file saved at: journal_yearly_ratios_with_counts.xlsx
